In [30]:
import pandas as pd
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
import numpy as np
import pickle

In [8]:
df = pd.read_csv('Churn_Modelling.csv')

In [9]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [10]:
df = df.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [11]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [13]:
le = LabelEncoder()
df['Gender'] = le.fit_transform(df['Gender'])

In [14]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [16]:
ohe = OneHotEncoder()
new_df = ohe.fit_transform(df[['Geography']])

In [22]:
new_df = pd.DataFrame(new_df.toarray(), columns=ohe.get_feature_names_out(['Geography']))

In [23]:
new_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [24]:
df = pd.concat(df.drop(['Geography'],axis=1),new_df)

TypeError: concat() takes 1 positional argument but 2 were given

In [26]:
df = df.drop(['Geography'],axis=1)

In [28]:
df = pd.concat([df,new_df],axis=1)

In [29]:
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [31]:
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(le,file)

with open('one_hotencoder.pkl','wb') as file:
    pickle.dump(ohe,file)

In [32]:
X = df.drop('Exited',axis=1)
y = df['Exited']

In [34]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [37]:
with open('Stscaler.pkl','wb') as file:
    pickle.dump(scaler,file)

In [49]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [46]:
model = Sequential(
    [
        Dense(64,activation='relu',input_shape=(X_train.shape[1],)),
        Dense(32,activation='relu'),
        Dense(1,activation='sigmoid')
    ]
)

In [47]:
opt = tf.keras.optimizers.legacy.Adam(learning_rate=0.01)
model.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])

In [50]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%y%m%d-%H%M%S")

In [51]:
tensorflow_callbacks = TensorBoard(log_dir=log_dir,histogram_freq=1)

In [52]:
early_stopping_callback = EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)

In [54]:
history = model.fit(
    X_train,y_train,validation_data = (X_test,y_test), epochs = 100,
    callbacks = [tensorflow_callbacks,early_stopping_callback]
)

Epoch 1/100
250/250 [==============================] - 1s 1ms/step - loss: 0.3981 - accuracy: 0.8342 - val_loss: 0.3703 - val_accuracy: 0.8460
Epoch 2/100
250/250 [==============================] - 0s 648us/step - loss: 0.3575 - accuracy: 0.8544 - val_loss: 0.3477 - val_accuracy: 0.8545
Epoch 3/100
250/250 [==============================] - 0s 703us/step - loss: 0.3475 - accuracy: 0.8572 - val_loss: 0.3448 - val_accuracy: 0.8595
Epoch 4/100
250/250 [==============================] - 0s 661us/step - loss: 0.3419 - accuracy: 0.8605 - val_loss: 0.3454 - val_accuracy: 0.8580
Epoch 5/100
250/250 [==============================] - 0s 654us/step - loss: 0.3415 - accuracy: 0.8570 - val_loss: 0.3583 - val_accuracy: 0.8520
Epoch 6/100
250/250 [==============================] - 0s 739us/step - loss: 0.3369 - accuracy: 0.8620 - val_loss: 0.3461 - val_accuracy: 0.8575
Epoch 7/100
250/250 [==============================] - 0s 669us/step - loss: 0.3350 - accuracy: 0.8631 - val_loss: 0.3475 - val_accu

In [55]:
model.save('model.h5')

/Users/sriharshavardhanperni/Desktop/calssif_problem/.venv/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [56]:
%load_ext tensorboard

In [57]:
%tensorboard --logdir logs/fit